<a href="https://colab.research.google.com/github/Prashanna-Raj-Pandit/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


# Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py

--2025-04-05 13:02:50--  https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10129 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.89K  --.-KB/s    in 0s      

2025-04-05 13:02:50 (51.6 MB/s) - ‘helper_functions.py’ saved [10129/10129]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

## Get a text dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2025-04-05 13:03:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   977KB/s    in 0.6s    

2025-04-05 13:03:05 (977 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing data
with the help of pandas


In [5]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# suffling train data
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [8]:
# how many samples?
len(train_df),len(test_df)

(7613, 3263)

In [9]:
# Lets visualize some random samples.
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","real disaster" if target>0 else "not real disaster")
  print(f"Text: {text}")
  print("_____")

Target:0 not real disaster
Text: This Attempted Mass Murder brought to You by the Obama Administration http://t.co/jbrK8ZsrY6
_____
Target:0 not real disaster
Text: @BaileySMSteach The fear of not looking like you know what you are doingÛ_ahhhÛ_that was a big one for me.  #PerryChat
_____
Target:0 not real disaster
Text: Build your own kingdom and lead your army to victory! https://youtu.

Start g this friend code: LZKTJNOX http://t.co/zZ0cEwEw64
_____
Target:0 not real disaster
Text: AD Miles 'Hurricane of Fun : The Making of Wet Hot' https://t.co/SBZwRuwuFh
_____
Target:1 real disaster
Text: Football hooligan jailed for rioting before game in Scotland was already banned from matches in England #UkNews http://t.co/q5mp2Q6Hy8
_____


In [10]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


## split data in to training and validation set

In [11]:
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1, # use 10% of training data for validation
                                                                       random_state=42)

In [12]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [13]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text vectorizarion (Tokenization)
### Converting text into numbers

*   Tokenization: Tokenization is the process of breaking down text into smaller units called tokens, which can be words, subwords, or even characters. Its a direct mapping of token to a number
*   Embedding: Create a matrix of feature vector for each token.
Embeddings convert tokens (words/subwords) into numerical vectors so that machines can process them. They capture semantic meaning (e.g., "king" is closer to "queen" than to "apple").Contextual Embeddings (Modern NLP)

Words get different vectors based on context (e.g., BERT, GPT).



In [14]:
import tensorflow as tf
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=None,
                                                  standardize='lower_and_strip_punctuation',
                                                  split='whitespace',
                                                  ngrams=None, #create groups of n-words
                                                  output_mode='int',
                                                  output_sequence_length=None, # how long do u want your sequence to be?
                                                  pad_to_max_tokens=False
                                                  )


### find the average number of tokens or words in training samples

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# setup text vectorization variables
max_vocab_length=10000 # Maximum number of words to have in our vocabolary
max_length=15 #max length our sentences will be
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                  output_mode='int',
                                                  output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# lets create a sample sentences and tokenize it
sample_sentence="The recent earthquake has destroyed the entire city"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 2756,  290,   41,  351,    2,  855,  182,    0,    0,    0,
           0,    0,    0,    0]])>

In [19]:
# selecting a random sample from the train dataset and tokenize it
import random
random_sentence=random.choice(train_sentences)
print(f"Random Sample: {random_sentence}")
print(f"Vectorized sample: {text_vectorizer([random_sentence])}")

Random Sample: Stretcher-bearers were soldiers whose job was to follow behind the advance and bring the wounded back into their... http://t.co/yeRcT4J244
Vectorized sample: [[8021   67 1362 4249  704   23    5 1023 1039    2 2571    7  962    2
   269]]


In [20]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() # Get all the unique words in the training dataset
top_5_words=words_in_vocab[:5] # get the top 5 most common words
bottom_5_words=words_in_vocab[-5:] # get the least 5 common words
print(f"Words in Vocab: {len(words_in_vocab)}")
print(f"Top 5 words: {top_5_words}")
print(f"Least 5 words: {bottom_5_words}")

Words in Vocab: 10000
Top 5 words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Least 5 words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


### creating an Embedding using Embedding layer

*   input_dim=the size of our vocabolary
*   output_dim= the size of the output embedding vector
*   input_length= size of the sequences being passed to the embedding layer

Key Properties of an Embedding Layer

**Trainable:**

* The vectors are updated via backpropagation (like other weights in a neural network).

* Pretrained embeddings (e.g., GloVe) can also be fine-tuned.

**Efficiency:**
More efficient than one-hot encoding + dense layers (avokes high-dimensional sparse matrices).

**Interpretability:**
Learned embeddings often capture semantic relationships (e.g., similar words cluster together).


In [21]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length, # set input shape
                           output_dim=128, # output shape
                           input_length=max_length # how long is each input
                           )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
# embed the random sample. i.e turn into dense vector of fixed size
print("Random sample: ",random_sentence)
print("After Embedding:...................")
sample_embed=embedding(text_vectorizer([random_sentence]))
sample_embed

Random sample:  Stretcher-bearers were soldiers whose job was to follow behind the advance and bring the wounded back into their... http://t.co/yeRcT4J244
After Embedding:...................


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03389039,  0.01488275,  0.00944055, ..., -0.04681885,
         -0.03574947, -0.02301905],
        [ 0.0496296 ,  0.00959368, -0.00418476, ..., -0.04974725,
         -0.0272707 , -0.02141514],
        [-0.00767081, -0.00192059,  0.0387955 , ..., -0.02602031,
          0.04331306,  0.04929036],
        ...,
        [-0.02767305,  0.04323922,  0.03603251, ...,  0.03897447,
          0.01414337,  0.03103079],
        [ 0.00732101,  0.0409486 ,  0.03100637, ...,  0.01113687,
         -0.04874425,  0.04683647],
        [-0.00230206,  0.00097422, -0.03826306, ..., -0.00167607,
         -0.033809  ,  0.01621777]]], dtype=float32)>

In [23]:
# checkout single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03389039,  0.01488275,  0.00944055,  0.0006662 , -0.04399751,
        -0.03743224, -0.03172175,  0.02998245, -0.00902388, -0.03136694,
        -0.04820877, -0.00109495,  0.00515987, -0.03477311,  0.0460845 ,
         0.01007   , -0.00502713,  0.00412413,  0.01304749, -0.03524045,
         0.00268512, -0.01688887, -0.0132182 ,  0.03373903, -0.03333769,
        -0.03153193, -0.00377398,  0.01388885,  0.02930875,  0.00857735,
        -0.04067909,  0.00030039, -0.02134645, -0.04816651, -0.02995712,
        -0.01077677,  0.04751964, -0.02014877, -0.01795517, -0.02000841,
         0.03319812, -0.04247948, -0.02339768, -0.01500529, -0.03678853,
         0.00085118,  0.03577537, -0.02743635, -0.01806276, -0.00646831,
         0.008931  , -0.03901292, -0.0445853 ,  0.04089722, -0.03593646,
        -0.01613258, -0.01330744,  0.00622297, -0.04982125,  0.0405394 ,
        -0.0227052 ,  0.04571625,  0.00493098, -0.00752817,  0.02169085,
  

This is the embedding for the first word of sentence

# Model 0: creating a baseline
To create baseline, let's use Sklearn Multimodal Naive Bayes using TF-IDF formula to convert text into numbers.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# create tokenization and modeling pipeline
model_0=Pipeline([ # pipeline is like keras Sequantial here
    ('tfidf',TfidfVectorizer()), # convert words into numbers using tfidf
    ('clf',MultinomialNB()), #model the text
])
# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

## Evaluate baseline model

In [25]:
baseline_accuracy=model_0.score(val_sentences,val_labels)
print(f"Model 0 (Baseline) Accuracy: {baseline_accuracy*100:.3f} %")

Model 0 (Baseline) Accuracy: 79.265 %


In [26]:
baseline_preds=model_0.predict(val_sentences)
print("Predicted labels")
baseline_preds[:20]

Predicted labels


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
print("Actual labels")
val_labels[:20]

Actual labels


array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [28]:
def evaluate_model(model,true_labels,predicted_labels):
  precision=tf.keras.metrics.Precision()
  precision.update_state(true_labels,predicted_labels)
  recall=tf.keras.metrics.Recall()
  recall.update_state(true_labels,predicted_labels)
  f1=(2*(precision.result()*recall.result()))/(precision.result()+recall.result())
  print(f"Precision: {precision.result()*100:.2f}%")
  print(f"Recall: {recall.result()*100:.2f}%")
  print(f"F1 score: {f1*100:.2f}%")

evaluate_model(model_0,val_labels,baseline_preds)



Precision: 88.62%
Recall: 62.64%
F1 score: 73.40%


In [29]:
# from sk-learn
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_performance_metrics(true_labels,predicted_labels):
  accuracy=accuracy_score(true_labels,predicted_labels)
  precision_score,recall_score,f1_score,_=precision_recall_fscore_support(true_labels,predicted_labels,average="weighted")
  model_results={
      "accuracy":accuracy,
      "precision":precision_score,
      "recall":recall_score,
      "f1_score":f1_score
  }
  return model_results

In [30]:
model_0_performance=calculate_performance_metrics(val_labels,baseline_preds)
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

The two evaluation methods (evaluate_model and calculate_performance_metrics) produce different results because:
* Different Libraries & Implementations

tf.keras.metrics (TensorFlow) and sklearn.metrics (scikit-learn) compute metrics differently.

Example:

>tf.keras.metrics.Precision() uses micro-averaging by default.

>sklearn.metrics.precision_recall_fscore_support() uses weighted averaging (as specified by average="weighted").
Averaging Method

* Weighted vs. Micro Averaging:

>Weighted (scikit-learn): Accounts for class imbalance by weighting metrics by class support.

>Micro (TensorFlow): Aggregates all classes equally, which can skew results if classes are imbalanced.

# Model 1: Simple dense model

In [31]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback
# create a directory to save the tensorboard logs
SAVE_DIR="mdoel_logs"

In [32]:
# Building a model with functional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string") # inputs are one dimentional string
x=text_vectorizer(inputs) # turn input text into number
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # lower the dimentionality of the embedding
outputs=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                          callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_1_dense")])


Saving TensorBoard log files to: mdoel_logs/model_1_dense/20250405-130310
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.6476 - loss: 0.6506 - val_accuracy: 0.7375 - val_loss: 0.5406
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8022 - loss: 0.4710 - val_accuracy: 0.7861 - val_loss: 0.4708
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8619 - loss: 0.3531 - val_accuracy: 0.7913 - val_loss: 0.4596
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.8917 - loss: 0.2900 - val_accuracy: 0.7874 - val_loss: 0.4686
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9103 - loss: 0.2393 - val_accuracy: 0.7835 - val_loss: 0.4791


In [35]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7713 - loss: 0.5112


[0.4791279435157776, 0.7834645509719849]

In [36]:
# make predictions
model_1_pred_probability=model_1.predict(val_sentences)
model_1_pred_probability.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


(762, 1)

In [37]:
model_1_pred_probability[0]

array([0.37831894], dtype=float32)

In [38]:
model_1_pred_probability=tf.squeeze(tf.round(model_1_pred_probability))
model_1_pred_probability[:50]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.],
      dtype=float32)>

In [39]:
val_labels[:50]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0])

In [40]:
model_1_perfornamce=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_1_pred_probability)
model_1_perfornamce

{'accuracy': 0.7834645669291339,
 'precision': 0.7872123378365872,
 'recall': 0.7834645669291339,
 'f1_score': 0.7807800582578169}

In [41]:
# compare with the baseline
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

Here, Our base line model outperforms the Deep Learning Model 1

# Visualizing embedding layer

In [42]:
len(words_in_vocab), words_in_vocab[:10] # the most common words

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [43]:
# get the weight matrix of embedding layer
embeded_weights=model_1.get_layer("embedding").get_weights()[0]
embeded_weights

array([[-0.05264153, -0.04492714,  0.00062678, ..., -0.06165594,
        -0.00452254,  0.0444353 ],
       [ 0.02477836,  0.02167839,  0.04398128, ...,  0.00505115,
        -0.03237456,  0.02942681],
       [-0.00428112,  0.028528  ,  0.0427485 , ..., -0.00014649,
        -0.0600719 ,  0.05856503],
       ...,
       [-0.04175667,  0.0376262 , -0.0354021 , ...,  0.01364893,
        -0.04660067, -0.02400279],
       [ 0.00347592, -0.06161162,  0.09168403, ..., -0.073577  ,
        -0.08855716,  0.03556163],
       [-0.02325767, -0.03214432,  0.05946862, ..., -0.04737134,
        -0.03754548,  0.02283878]], dtype=float32)

In [44]:
embeded_weights.shape

(10000, 128)

## Embedding Projector

In [45]:
# Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format:
# a file of vectors (containing the embedding), and a file of meta data (containing the words).
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embeded_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [46]:
# If you are running this tutorial in Colaboratory, you can use the following snippet to download these files to your local machine
#  (or use the file browser, View -> Table of contents -> File browser).
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model: 2
LSTM : Long short term memory
> Input->Tokenizer->Embeddings->Layers(RNN/Dense)->Output(label probability)

In [50]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),name="input_layer",dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)
# x=layers.LSTM(units=64,return_sequences=True)(x) # when we are stacking the LSTM cells we need to set the return_sequences= true
print(x.shape)
x=layers.LSTM(64)(x)
print(x.shape)
# x=layers.Dense(64,activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)

model_2=tf.keras.Model(inputs,outputs,name="model_2_LSTM")

(None, 15, 128)
(None, 15, 128)
(None, 64)


In [51]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
# compile the mdoel
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [53]:
# fit the model
model_2_history=model_2.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_2_lstm")])

Saving TensorBoard log files to: mdoel_logs/model_2_lstm/20250405-133035
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.9013 - loss: 0.2972 - val_accuracy: 0.7848 - val_loss: 0.5325
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.9530 - loss: 0.1413 - val_accuracy: 0.7822 - val_loss: 0.5963
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9557 - loss: 0.1201 - val_accuracy: 0.7808 - val_loss: 0.6606
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.9588 - loss: 0.1054 - val_accuracy: 0.7822 - val_loss: 0.7160
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.9665 - loss: 0.0877 - val_accuracy: 0.7756 - val_loss: 0.8473


In [58]:
# make prediction
model_2_pred=model_2.predict(val_sentences)
model_2_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


array([[7.3352954e-03],
       [7.7080947e-01],
       [9.9965346e-01],
       [4.1415494e-02],
       [5.8285287e-04],
       [9.8837590e-01],
       [6.7433244e-01],
       [9.9976552e-01],
       [9.9955863e-01],
       [3.2206753e-01]], dtype=float32)

In [59]:
model_2_pred_probability=tf.squeeze(tf.round(model_2_pred))
model_2_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

## Model 2 vs Model 1 vs Model 0

Still our baseline beats our Model 1 and Model 0

In [60]:
model_2_performance=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_2_pred_probability)
model_2_performance

{'accuracy': 0.7755905511811023,
 'precision': 0.7810725814480683,
 'recall': 0.7755905511811023,
 'f1_score': 0.7720383798735818}

In [61]:
model_1_perfornamce

{'accuracy': 0.7834645669291339,
 'precision': 0.7872123378365872,
 'recall': 0.7834645669291339,
 'f1_score': 0.7807800582578169}